In [5]:
!git clone -b fix_object_sfl https://github.com/eaguaida/RISE-SFL.git

fatal: destination path 'RISE-SFL' already exists and is not an empty directory.


In [6]:
!ls -a

.  ..  .config	RISE-SFL  sample_data


In [7]:
import sys
import os

rise_sfl_path = "/content/RISE-SFL/RISE-SFL"  # Replace with the actual path to your RISE-SFL folder
sys.path.append(rise_sfl_path)

In [8]:
from utils.utils import *
# From masker/generation.py
from masker.generation import SFL

ModuleNotFoundError: No module named '_innit_'

In [ ]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms
from PIL import Image
from keras.preprocessing import image
# Assuming utils, evaluation, and explanations modules are custom and present in your project
import json
from torch.nn.functional import softmax, conv2d

from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
cudnn.benchmark = True
# Load black box model for explanations
model = models.resnet50(True)
model = nn.Sequential(model, nn.Softmax(dim=1))
model = model.eval()
model = model.cuda()

for p in model.parameters():
    p.requires_grad = False

# To use multiple GPUs
model = nn.DataParallel(model)

In [ ]:
import os
import requests
import zipfile
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import numpy as np

In [ ]:
'''
img_path = '/content/RISE-SFL/data/catdog.png'
img = read_tensor(img_path)

with torch.no_grad():
    output = model(img)
    top_probabilities, top_classes = torch.topk(output, k=3, dim=1)
    top_pred_class = top_classes[0][1].item()
    top_pred_class_name = get_class_name(top_pred_class)

for zx in range(3):
    class_name = get_class_name(top_classes[0][zx].item())
    percentage = top_probabilities[0][zx].item() * 100
    print(f'Top {zx+1} class: {class_name}, Probability: {percentage:.2f}%')
'''

In [ ]:
#explainer = SFL(model, (224, 224))
from masker.batch import SFL_batch
input_size = (224,224)
sfl_batch = SFL_batch(model, input_size)
image_folder = '/content/RISE-SFL/data_demo'
masks, sampled_tensor = sfl.generate_mutants_batch()
N = 10
s = 8
p1 = 0.2
masks, sampled_tensors = sfl_batch.generate_batch_images(image_folder, N, s, p1)

In [ ]:
print(sampled_tensors[0].shape)
print(masks.shape)

In [ ]:
def generate_pass_fail_list(N):
    return ["Pass" if i % 2 == 0 else "Fail" for i in range(N)]

pass_fail_list = generate_pass_fail_list(N)

from visuals.plots import SFLVisualizer

In [ ]:
visualizer = SFLVisualizer()
visualizer.interactive_mutant_visualization(sampled_tensors[2], pass_fail_list)

In [ ]:
from sfl_techniques.sfl import RelevanceScore

relevance_score_calculator = RelevanceScore(device='cuda')
pixel_dataset = relevance_score_calculator.run(sampled_tensors[0], masks[0], N)

import

In [ ]:
from from visuals.saliency_maps.saliency import SaliencyMapVisualizer
saliency_map = SaliencyMapVisualizer('catdog.png')


In [ ]:
'''
def visualize_pixel_scores(dataset, image_shape, original_image_array):
    # Extract scores and reshape them into 2D arrays
    scores = {
        'Ep': np.zeros(image_shape),
        'Ef': np.zeros(image_shape),
        'Np': np.zeros(image_shape),
        'Nf': np.zeros(image_shape),
        'ochiai': np.zeros(image_shape),
        'tarantula': np.zeros(image_shape),
        'zoltar': np.zeros(image_shape),
        'wong1': np.zeros(image_shape)
    }
    for pixel in dataset:
        i, j = pixel['position']
        for score_type in scores.keys():
            scores[score_type][i, j] = pixel[score_type]

    # Invert values for ochiai and tarantula
    scores['ochiai'] = 1 - scores['ochiai']
    scores['tarantula'] = 1 - scores['tarantula']
    scores['zoltar']  = 1 - scores['zoltar']
    scores['wong1']  = 1 - scores['wong1']

    # Set up the plot
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))
    fig.suptitle("Pixel-wise Scores Visualization", fontsize=16)
    # Define a color map
    cmap = plt.get_cmap('jet')
    # Plot each score type
    for idx, (score_type, score_data) in enumerate(scores.items()):
        row = idx // 4
        col = idx % 4
        ax = axs[row, col]

        # Create the heatmap
        im = ax.imshow(original_image_array, cmap='gray', alpha=1)
        im = ax.imshow(score_data, cmap=cmap, alpha=0.5)
        ax.set_title(score_type)
        ax.axis('off')
        # Add colorbar
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im, cax=cax)
    # Adjust layout and display
    plt.tight_layout()
    plt.show()

# Load and resize the original image, and convert to grayscale


original_image = Image.open('/content/RISE-SFL/data_demo/catdog.png').resize((224, 224)).convert('L')
original_image_array = np.array(original_image)

visualize_pixel_scores(pixel_dataset, (224, 224), original_image_array)
'''